In [1]:
from nuplan.database.nuplan_db_orm.nuplandb import NuPlanDB
from pathlib import Path
from nuplan.database.nuplan_db_orm.nuplandb_wrapper import NuPlanDBWrapper
from nuplan.database.nuplan_db_orm.frame import Frame
import os

from nuplan.planning.utils.multithreading.worker_pool import Task
from nuplan.planning.utils.multithreading.worker_parallel import SingleMachineParallelExecutor
from nuplan.planning.scenario_builder.scenario_filter import ScenarioFilter
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_builder import NuPlanScenarioBuilder
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_utils import ScenarioMapping

import os
import math
import argparse
import matplotlib.pyplot as plt
from tqdm import tqdm
from data_utils import *
from trajectory_tree_planner import *
from common_utils import get_filter_parameters, get_scenario_map
from data_process import DataProcessor

In [2]:
db_path = "/cailiu2/Diffusion-Planner/data/2021.10.21.14.43.30_veh-28_01244_01519.db" # single db file
db_path = "/share/data_cold/open_data/nuplan/data/cache/mini"  # multi db files
data_root = "/cailiu2/Diffusion-Planner/data/"
map_path = "/cailiu2/Diffusion-Planner/data/maps"
map_version = "nuplan-maps-v1.0"

save_processed_path = "/cailiu2/Diffusion-Planner/data/processed"

In [3]:
total_scenarios_num = 10
scenario_mapping = ScenarioMapping(scenario_map=get_scenario_map(), subsample_ratio_override=0.5)
builder = NuPlanScenarioBuilder(db_path, map_path, None, None, map_version, scenario_mapping=scenario_mapping)
scenario_filter = ScenarioFilter(*get_filter_parameters(
                                                        num_scenarios_per_type=30000,
                                                        limit_total_scenarios=total_scenarios_num))

print(scenario_filter) # note: the <scenario_types> in <scenario_filter> is the scenario types thich you want
worker = SingleMachineParallelExecutor(use_process_pool=True)
scenarios = builder.get_scenarios(scenario_filter, worker)
print(f"Total number of training scenarios: {len(scenarios)}")
del worker, builder, scenario_filter

ScenarioFilter(scenario_types=['starting_left_turn', 'starting_right_turn', 'starting_straight_traffic_light_intersection_traversal', 'high_lateral_acceleration', 'high_magnitude_speed', 'low_magnitude_speed', 'traversing_pickup_dropoff', 'near_multiple_vehicles', 'changing_lane', 'following_lane_with_lead'], scenario_tokens=None, log_names=None, map_names=None, num_scenarios_per_type=30000, limit_total_scenarios=10, timestamp_threshold_s=None, ego_displacement_minimum_m=None, expand_scenarios=False, remove_invalid_goals=False, shuffle=False, ego_start_speed_threshold=None, ego_stop_speed_threshold=None, speed_noise_tolerance=None, token_set_path=None, fraction_in_token_set_threshold=None, ego_route_radius=None)
Total number of training scenarios: 10


Total number of training scenarios: 10


In [4]:
scenario_list = [scenario for scenario in scenarios[:10] ]
processor = DataProcessor(scenario_list)#extract the 0 scenarios
# processor.work(save_processed_path, debug = True)

In [5]:
processor.scenario = processor._scenarios[5]
ego_future_traj = processor.get_ego_agent_future()
print(ego_future_traj.shape)

ego_agent_past, time_stamps_past = processor.get_ego_agent()
neighbor_agents_past, neighbor_agents_types = processor.get_neighbor_agents()
ego_agent_past, neighbor_agents_past, neighbor_indices = \
        agent_past_process(ego_agent_past, time_stamps_past, neighbor_agents_past, neighbor_agents_types, processor.num_agents)



current_ego_state = processor.scenario.initial_ego_state
present_tracked_objects = processor.scenario.initial_tracked_objects.tracked_objects

future_tracked_objects = [
  tracked_objects.tracked_objects
  for tracked_objects in processor.scenario.get_future_tracked_objects(
      iteration=0, time_horizon=processor.future_time_horizon, num_samples=processor.num_future_poses
  )
]
sampled_future_observations = [present_tracked_objects] + future_tracked_objects
future_tracked_objects_tensor_list, _ = sampled_tracked_objects_to_tensor_list(sampled_future_observations)

agent_index = neighbor_indices

anchor_ego_state = torch.tensor([current_ego_state.rear_axle.x, current_ego_state.rear_axle.y, current_ego_state.rear_axle.heading,
                                     current_ego_state.dynamic_car_state.rear_axle_velocity_2d.x,
                                     current_ego_state.dynamic_car_state.rear_axle_velocity_2d.y,
                                     current_ego_state.dynamic_car_state.rear_axle_acceleration_2d.x,
                                     current_ego_state.dynamic_car_state.rear_axle_acceleration_2d.y])
agent_future = filter_agents_tensor(future_tracked_objects_tensor_list)


(80, 3)


In [6]:
print(agent_index)

[1, 4, 15, 2, 14, 13, 16, 6, 5, 8, 17, 3, 7, 0, 37, 38, 19, 20, 22, 28]


In [10]:
print(agent_future[1].shape)
local_coords_agent_states = []
for agent_state in agent_future:
  local_coords_agent_states.append(convert_absolute_quantities_to_relative(agent_future[1], anchor_ego_state, 'agent'))

print(local_coords_agent_states[0].shape)
padded_agent_states = pad_agent_states_with_zeros(local_coords_agent_states)
print(padded_agent_states[0].shape)

# fill agent features into the arrays
agent_futures = np.zeros(shape=(processor.num_agents, padded_agent_states.shape[0]-1, 3), dtype=np.float32)
for i, j in enumerate(agent_index):
    agent_futures[i] = padded_agent_states[1:, j, [AgentInternalIndex.x(), AgentInternalIndex.y(), AgentInternalIndex.heading()]].numpy()

print(agent_futures[0].shape)

print(padded_agent_states[0][:, -].shape)
print(padded_agent_states[0][0, :3])
print(agent_futures[0][0, :3])
print(padded_agent_states.shape)

torch.Size([42, 8])
torch.Size([42, 8])
torch.Size([42, 8])
(80, 3)
torch.Size([42, 3])
tensor([0.0000, 2.0255, 7.1906])
[ 1.4783206e+06 -5.8786119e+05  6.7113602e-01]
torch.Size([81, 42, 8])


In [58]:
object_types = [TrackedObjectType.VEHICLE, TrackedObjectType.PEDESTRIAN, TrackedObjectType.BICYCLE]
agents = future_tracked_objects[0].get_tracked_objects_of_types(object_types)
print(len(agents))

x_list = []
y_list = []
vx_list = []
vy_list = []
for i, agent in enumerate(agents):
    obj_state.append(agent.box.center.point)
    x_list.append(agent.center.point.x)
    y_list.append(agent.center.point.y)
    vx_list.append(agent.velocity.x)
    vy_list.append(agent.velocity.y)
p = figure(title="Torch Tensor Curve", x_axis_label="X", y_axis_label="Y", width=600, height=400)

# 绘制折线
p.line(x_list[:10], y_list[:10], line_width=2, color="blue", legend_label="Trajectory")

# 显示图像
show(p)

42


In [52]:
print(type(sampled_future_observations))
print(type(sampled_future_observations[1]))
print(list(sampled_future_observations[0]))

obj_state=[]
x_list = []
y_list = []
vx_list = []
vy_list = []
for i, agent in enumerate(sampled_future_observations[2]):
    obj_state.append(agent.box.center.point)
    x_list.append(agent.center.point.x)
    y_list.append(agent.center.point.y)
    vx_list.append(agent.velocity.x)
    vy_list.append(agent.velocity.y)
    
p = figure(title="Torch Tensor Curve", x_axis_label="X", y_axis_label="Y", width=600, height=400)

# 绘制折线
p.line(x_list[:10], y_list[:10], line_width=2, color="blue", legend_label="Trajectory")

# 显示图像
show(p)

print(vx_list)
print(vy_list)


<class 'list'>
<class 'nuplan.common.actor_state.tracked_objects.TrackedObjects'>
[<nuplan.common.actor_state.agent.Agent object at 0x7f3847aee5b0>, <nuplan.common.actor_state.agent.Agent object at 0x7f3847aee8e0>, <nuplan.common.actor_state.agent.Agent object at 0x7f3847aee7c0>, <nuplan.common.actor_state.agent.Agent object at 0x7f3847aeeaf0>, <nuplan.common.actor_state.agent.Agent object at 0x7f384e633b20>, <nuplan.common.actor_state.agent.Agent object at 0x7f384a50a250>, <nuplan.common.actor_state.agent.Agent object at 0x7f384a50a8e0>, <nuplan.common.actor_state.agent.Agent object at 0x7f384a50a790>, <nuplan.common.actor_state.agent.Agent object at 0x7f384a50abe0>, <nuplan.common.actor_state.agent.Agent object at 0x7f384a50afd0>, <nuplan.common.actor_state.agent.Agent object at 0x7f3847b0fa30>, <nuplan.common.actor_state.agent.Agent object at 0x7f38453df1f0>, <nuplan.common.actor_state.agent.Agent object at 0x7f38453df2b0>, <nuplan.common.actor_state.agent.Agent object at 0x7f38453d

[1.4586719518198938, -2.542753191041794, -1.8920678778772522, -1.0146386931287272, -2.2951299425560627, -4.430320672174632, -1.7781214456571095, -3.5260419644374035, -2.954418546178093, -5.617329385957783, 0.0, 0.0, 0.0, -2.138872239978283, -2.6929587673947455, -2.0762069135545147, -1.869295660171175, -0.16687370650532143, 0.0, 3.257659605245421, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.904097320385934, -3.703963853735043, 6.599387585772917, 0.0, 0.4462775414228296, 0.5239176090835659, 0.4806347969946071, 0.46665228602405523, 0.4805029904352532, 0.5156745282144222, -0.00040580054194594775, 0.5406834101488153, -0.5009528467662728, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0